In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data_dir = "/content/drive/MyDrive/farm_insects_project/data/processed/farm_insects"


train_dir = f"{data_dir}/splits/train"
val_dir = f"{data_dir}/splits/val"
test_dir = f"{data_dir}/splits/test"


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(test_dir, transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))

Classes: ['Africanized Honey Bees (Killer Bees)', 'Aphids', 'Armyworms', 'Brown Marmorated Stink Bugs', 'Cabbage Loopers', 'Citrus Canker', 'Colorado Potato Beetles', 'Corn Borers', 'Corn Earworms', 'Fall Armyworms', 'Fruit Flies', 'Spider Mites', 'Thrips', 'Tomato Hornworms', 'Western Corn Rootworms']
Number of training images: 808


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
#full-fine tuning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

resnet50 = resnet50.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=1e-5)

num_epochs = 20

for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")


resnet50.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet50(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("\n Final Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 179MB/s]


Epoch [1/20] - Training Loss: 2.6595
Epoch [2/20] - Training Loss: 2.4009
Epoch [3/20] - Training Loss: 2.1668
Epoch [4/20] - Training Loss: 1.9234
Epoch [5/20] - Training Loss: 1.7063
Epoch [6/20] - Training Loss: 1.5164
Epoch [7/20] - Training Loss: 1.3218
Epoch [8/20] - Training Loss: 1.1877
Epoch [9/20] - Training Loss: 1.0468
Epoch [10/20] - Training Loss: 0.9245
Epoch [11/20] - Training Loss: 0.8365
Epoch [12/20] - Training Loss: 0.7469
Epoch [13/20] - Training Loss: 0.6507
Epoch [14/20] - Training Loss: 0.6064
Epoch [15/20] - Training Loss: 0.5068
Epoch [16/20] - Training Loss: 0.4705
Epoch [17/20] - Training Loss: 0.4133
Epoch [18/20] - Training Loss: 0.3650
Epoch [19/20] - Training Loss: 0.3344
Epoch [20/20] - Training Loss: 0.3009

 Final Performance Metrics:
Accuracy : 0.7154
Precision: 0.7245
Recall   : 0.7154
F1 Score : 0.7140


In [7]:
#partial fine-tuning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.layer4.parameters():
    param.requires_grad = True

num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

resnet50 = resnet50.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet50.parameters()), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")


resnet50.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet50(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("\n Final Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

Using device: cpu
Epoch [1/20] - Training Loss: 2.0478
Epoch [2/20] - Training Loss: 0.9512
Epoch [3/20] - Training Loss: 0.5042
Epoch [4/20] - Training Loss: 0.2981
Epoch [5/20] - Training Loss: 0.1701
Epoch [6/20] - Training Loss: 0.1305
Epoch [7/20] - Training Loss: 0.0912
Epoch [8/20] - Training Loss: 0.0869
Epoch [9/20] - Training Loss: 0.1134
Epoch [10/20] - Training Loss: 0.0787
Epoch [11/20] - Training Loss: 0.0696
Epoch [12/20] - Training Loss: 0.0825
Epoch [13/20] - Training Loss: 0.0591
Epoch [14/20] - Training Loss: 0.0573
Epoch [15/20] - Training Loss: 0.0600
Epoch [16/20] - Training Loss: 0.0659
Epoch [17/20] - Training Loss: 0.0515
Epoch [18/20] - Training Loss: 0.0675
Epoch [19/20] - Training Loss: 0.0937
Epoch [20/20] - Training Loss: 0.0618

 Final Performance Metrics:
Accuracy : 0.6957
Precision: 0.7296
Recall   : 0.6957
F1 Score : 0.6946


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#static

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

for param in resnet50.parameters():
    param.requires_grad = False


num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

resnet50 = resnet50.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.fc.parameters(), lr=1e-3)

num_epochs = 20

for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")


resnet50.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet50(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("\n Final Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 166MB/s]


Epoch [1/20] - Training Loss: 2.3582
Epoch [2/20] - Training Loss: 1.6144
Epoch [3/20] - Training Loss: 1.2812
Epoch [4/20] - Training Loss: 1.1359
Epoch [5/20] - Training Loss: 1.0122
Epoch [6/20] - Training Loss: 0.8937
Epoch [7/20] - Training Loss: 0.8368
Epoch [8/20] - Training Loss: 0.7874
Epoch [9/20] - Training Loss: 0.7708
Epoch [10/20] - Training Loss: 0.7034
Epoch [11/20] - Training Loss: 0.6604
Epoch [12/20] - Training Loss: 0.6562
Epoch [13/20] - Training Loss: 0.5899
Epoch [14/20] - Training Loss: 0.6017
Epoch [15/20] - Training Loss: 0.5409
Epoch [16/20] - Training Loss: 0.5367
Epoch [17/20] - Training Loss: 0.5437
Epoch [18/20] - Training Loss: 0.4655
Epoch [19/20] - Training Loss: 0.4627
Epoch [20/20] - Training Loss: 0.4666

 Final Performance Metrics:
Accuracy : 0.6245
Precision: 0.6596
Recall   : 0.6245
F1 Score : 0.6167
